# 02 · Trend Analysis — Baltimore IGS
# -----------------------------------
# Safe to run after 01_ingest_and_eda.ipynb
# Requires: pandas, numpy, matplotlib (no parquet engines needed)

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Paths ---
ROOT = Path.cwd().parents[0]  # project root (igs-analysis-baltimore)
CLEAN_DIR = ROOT / "data_clean"
VIS_DIR = ROOT / "visuals"
VIS_DIR.mkdir(parents=True, exist_ok=True)

merged_path = CLEAN_DIR / "baltimore_igs_merged.csv"
deltas_path = CLEAN_DIR / "baltimore_igs_yoy_deltas.csv"

# --- Load data ---
print("Looking for cleaned files in:", CLEAN_DIR.resolve())
assert merged_path.exists(), f"❌ Missing cleaned file: {merged_path}"
merged = pd.read_csv(merged_path)

if deltas_path.exists():
    deltas = pd.read_csv(deltas_path)
    print("Loaded deltas file.")
else:
    deltas = None
    print("⚠️ No deltas file found — will compute YoY internally.")

print("✅ Loaded merged dataset with shape:", merged.shape)
merged.head()
